# Design with explicit Formula

This tutorial notebook shows how to setup a D-optimal design with BoFire while providing an explicit formula and not just one of the four available keywords `linear`, `linear-and-interaction`, `linear-and-quadratic`, `fully-quadratic`.

Make sure that `cyipopt`is installed. The recommend way is the installation via conda `conda install -c conda-forge cyipopt`.

## Imports

In [ ]:
from formulaic import Formula

from bofire.data_models.api import Domain, Inputs
from bofire.data_models.features.api import ContinuousInput
from bofire.strategies.doe.design import find_local_max_ipopt
from bofire.utils.doe import get_confounding_matrix

## Setup of the problem

In [ ]:
input_features = Inputs(
    features=[
        ContinuousInput(key="a", bounds=(0, 5)),
        ContinuousInput(key="b", bounds=(40, 800)),
        ContinuousInput(key="c", bounds=(80, 180)),
        ContinuousInput(key="d", bounds=(200, 800)),
    ],
)
domain = Domain(inputs=input_features)

## Definitionn of the formula for which the optimal points should be found

In [ ]:
model_type = Formula("a + {a**2} + b + c + d + a:b + a:c + a:d + b:c + b:d + c:d")
model_type

## Find D-optimal Design

In [ ]:
design = find_local_max_ipopt(domain=domain, model_type=model_type, n_experiments=17)
design

## Analyze Confounding

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


matplotlib.rcParams["figure.dpi"] = 120

m = get_confounding_matrix(
    domain.inputs,
    design=design,
    interactions=[2, 3],
    powers=[2],
)

sns.heatmap(m, annot=True, annot_kws={"fontsize": 7}, fmt="2.1f")
plt.show()